<a id='top'></a>
<h1>PISM paleo data to zarr </h1>

Transformation of data from Antarctic Ice Sheet glacial cycle experiments

(c) 2021 by Torsten Albrecht (torsten.albrecht@pik-potsdam.de) | Potsdam Institute for Climate Impact Research (PIK)

based on https://github.com/ldeo-glaciology/pangeo-bedmachine
and https://github.com/ldeo-glaciology/pangeo-pismpaleo


*data:* Albrecht, Torsten (2018): PISM simulation results of the Antarctic Ice Sheet deglaciation. GFZ Data Services. https://doi.org/10.5880/PIK.2018.008


Supplement to: Kingslake, J., Scherer, R. P., Albrecht, T., Coenen, J., Powell, R. D., Reese, R., … Whitehouse, P. L. (2018). Extensive retreat and re-advance of the West Antarctic Ice Sheet during the Holocene. Nature. doi:10.1038/s41586-018-0208-x


<a id='overview'></a>
<h2>Overview of settings</h2>

**Forcings and initial values** see https://github.com/pism/pism-ais

*Initial ice configuration:* Bedmap2, Fretwell et al., 2013

*Atmosphere:* RACMO v2.1, HadCM3; Ligtenberg et al., 2013

*Ocean:* Schmidtko et al. 2014, Zwally et al., 2012

*Basal Heatflux:* Shapiro and Ritzwoller, 2014



**Key parameters**
```

ecalv = 1e17 m s
hcalv = 75 m
gammaT = 1.0e-5
overturning_coeff = 0.8e6
ppq = 0.75
essa = 0.6
esia = 2.0
till_effo = 0.04
till_dec = 1 mm/yr
pscale = 2 %/K
visc = 0.5e21 Pa s

```

**Run length**

35,000 years (after spin-up over 205,000 years)

**Horizontal grid resolution**

15000 m x 15000 m (400 x 400 grid points)

<br>
<div class="alert alert-block alert-warning">**PISM code version**
<br><br>
PISMR (development c0cbb7b committed by Torsten Albrecht on 2017-04-06 12:13:27 +0200<br>
see branch <a href="https://github.com/pism/pism/tree/pik/paleo_07dev">https://github.com/pism/pism/tree/pik/paleo_07dev</a>,<br>
see release <a href="https://github.com/pism/pism/releases/tag/pik-holocene-gl-rebound">https://github.com/pism/pism/releases/tag/pik-holocene-gl-rebound</a>,<br>
published in <a href="https://
doi.org/10.5281/zenodo.1199066">https://
doi.org/10.5281/zenodo.1199066</a><br>
</div>


pism/pik/paleo_07dev

<a id='settings'></a>
<h2>Notebook settings</h2>

In [1]:
%%javascript
// hide presentation toolbar
$('.nbp-app-bar').toggle()

// disable autoscroll in output cells
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Make the notebook cells take almost all available width
# More info:
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

<a id='constants'></a>
<h2>PISM constants and pathes</h2>

In [3]:
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

import matplotlib.pylab as plt
#from matplotlib import cm, colors, rcParams
import netCDF4 as nc
import numpy as np
#import copy
#import os, glob, sys, copy
#import pandas as pd
import xarray as xr
import gcsfs
import json
#import warnings
import cftime
import copy, os
from tqdm.notebook import tqdm   # progress bar

### set constants
seconds_per_year = 365.0*24.0*60.0*60.0 #3.1556926e7

# public
trunk = "/p/projects/tumble/albrecht/data_rebound/"
datapath = trunk+"datapub/model_data/"

# PIK archive
datapath = trunk+"archive/model_data/"


xr.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
python-bits: 64
OS: Linux
OS-release: 4.4.162-94.72-default
machine: x86_64
processor: x86_64
byteorder: little
LC_ALL: None
LANG: en_US.UTF-8
LOCALE: en_US.UTF-8
libhdf5: 1.10.4
libnetcdf: 4.7.3

xarray: 0.16.1
pandas: 1.1.4
numpy: 1.19.4
scipy: 1.5.2
netCDF4: 1.5.3
pydap: None
h5netcdf: None
h5py: None
Nio: None
zarr: 2.5.0
cftime: 1.3.0
nc_time_axis: None
PseudoNetCDF: None
rasterio: None
cfgrib: None
iris: None
bottleneck: None
dask: 2.30.0
distributed: 2.30.1
matplotlib: 3.3.3
cartopy: 0.18.0
seaborn: 0.11.0
numbagg: None
pint: None
setuptools: 50.3.0.post20201006
pip: 20.2.4
conda: None
pytest: None
IPython: 7.18.1
sphinx: None


<a id='pismfiles'></a>
<h2>Collect reference simulation results in dict</h2>

In [4]:

refrun = {}
refrun['exp_dir'] = datapath
refrun['run_id'] = 2205
refrun['exp_name'] = refrun['exp_dir']+"forcing"+str(refrun['run_id'])+"_LGM/results"
refrun['notes'] = "PISM paleo reference simulation data as in https://doi.org/10.5880/PIK.2018.008"

model_files = ["present","extra"] #,"timeseries"]
refrun['file_path'] = dict.fromkeys(model_files,"")
        
#present_file = "result_forcing_15km_205000yrs_basal.nc"
present_file = "result_forcing_15km_205000yrs.nc"
refrun['file_path']['present'] = [os.path.join(refrun['exp_name'], present_file)]
        
#extra_file = "extra_forcing_15km_205000yrs.nc"
extra_file = "extra_forcing_15km_205000yrs_tozarr.nc"
refrun['file_path']['extra'] = [os.path.join(refrun['exp_name'], extra_file)]
        
time_file = "ts_forcing_15km_205000yrs.nc"
refrun['file_path']['timeseries'] = [os.path.join(refrun['exp_name'], time_file)]


<a id='xarrayload'></a>
<h2>Load PISM netCDF files to xarrays </h2>

In [5]:

variables={#'present':['mask','velsurf_mag'],
           #'extra':['mask','thk','topg','usurf'],
           'present':['mask','thk','topg','usurf','velbar_mag','dbdt'],
           'extra':['mask','thk','topg','usurf','velbar_mag'],
           'timeseries':['slvol','volume_glacierized','time']}

datasets=[]
for mf in tqdm(model_files):
        print(mf)
        #datasets=[]
        dataset=[]
        if refrun['file_path'][mf]:
            for pismf in refrun['file_path'][mf]:
              #with xr.set_options(enable_cftimeindex=True):
                with xr.open_dataset(pismf, decode_cf=True,decode_times=False,use_cftime=True,chunks={'x': 400, 'y': 400, 'time': -1}) as ds: #decode_timedelta=True
                    ds = ds.get(variables[mf])
                    # convert to years
                    ds["time"]=(['time'], ds["time"].time/seconds_per_year)

                    dataset.append(ds)
                    
        concat_time = xr.concat(dataset, dim='time') #,data_vars=list([u'mask',u'thk'])
        datasets.append(concat_time)


present
extra



In [6]:
datasets

[<xarray.Dataset>
 Dimensions:     (time: 1, x: 400, y: 400)
 Coordinates:
   * time        (time) float64 0.0
   * x           (x) float64 -2.795e+06 -2.78e+06 ... 3.175e+06 3.19e+06
   * y           (y) float64 -2.795e+06 -2.78e+06 ... 3.175e+06 3.19e+06
     lat         (y, x) float64 dask.array<chunksize=(400, 400), meta=np.ndarray>
     lon         (y, x) float64 dask.array<chunksize=(400, 400), meta=np.ndarray>
 Data variables:
     mask        (time, y, x) int8 dask.array<chunksize=(1, 400, 400), meta=np.ndarray>
     thk         (time, y, x) float64 dask.array<chunksize=(1, 400, 400), meta=np.ndarray>
     topg        (time, y, x) float64 dask.array<chunksize=(1, 400, 400), meta=np.ndarray>
     usurf       (time, y, x) float64 dask.array<chunksize=(1, 400, 400), meta=np.ndarray>
     velbar_mag  (time, y, x) float32 dask.array<chunksize=(1, 400, 400), meta=np.ndarray>
     dbdt        (time, y, x) float64 dask.array<chunksize=(1, 400, 400), meta=np.ndarray>
 Attributes:
     C

<a id='xarraytozarr'></a>
<h2>Export to Zarr Format</h2>

In [7]:
#import subprocess
#
#def du(path):
#    """disk usage in human readable format (e.g. '2,1GB')"""
#    return subprocess.check_output(['du','-sh', path]).split()[0].decode('utf-8')


for mi,mf in enumerate(tqdm(model_files)):
    print(mi,mf)
    zarr_present_folder = "/p/tmp/albrecht/paleo_ensemble/pangeo/reference18/"+mf
    #if os.path.exists(zarr_present_folder):
    #    os.removedirs(zarr_present_folder)
    datasets[mi].to_zarr(zarr_present_folder, consolidated=True, mode='w')
    #print(du(zarr_present_folder),commands.getoutput('du -sh '+zarr_present_folder).split()[0])
 

# size of zarr folders:
# public
# 348M	extra
# 3M	present

# archive
# 407M	extra
# 7M	present


0 present
1 extra



<a id='addtoken'></a>
<h2>Add token from LDEO which enables to write to bucket </h2>


In [8]:
#print(gcsfs.__version__)
with open('../tokens/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)
gcs = gcsfs.GCSFileSystem(token=token, access='read_write')

<a id='writezarr'></a>
<h2>Save zarr to bucket </h2>


In [9]:
for mi,mf in enumerate(tqdm(model_files)):
    print(mi,mf)
    #alldatasets[mi] = alldatasets[mi].drop('mapping')   # remove the variable.
    mapper = gcs.get_mapper('gs://ldeo-glaciology/paleo_ensemble/reference18_'+mf+'.zarr')
    datasets[mi].to_zarr(mapper, consolidated=True, mode='w')

0 present
1 extra



<a id='checkzarr'></a>
<h2>Reload the zarr and compare with original </h2>


In [10]:
present_reloaded = xr.open_zarr(gcs.get_mapper('gs://ldeo-glaciology/paleo_ensemble/reference18_present.zarr'))  
present_reloaded.identical(datasets[0])

True

<a id='testplot'></a>
<h2>Do some simple plot</h2>

In [11]:
#present_reloaded.mask.mean(dim='id').plot()

In [12]:
#cp ~/python/jupyter/paleo/pism_reference18_nc_to_zarr.ipynb ~/www/notebooks/paleo_paper/
#see https://nbviewer.jupyter.org/url/www.pik-potsdam.de/~albrecht/notebooks/paleo_paper/pism_reference18_nc_to_zarr.ipynb